# DICOM Standard harvesting of Part 3 
## Extract relationships between Tags and Value Sets

[Useful Online XML Viewer](https://jsonformatter.org/xml-viewer)

Links to the XML Objects
- [DICOM Part 3](https://dicom.nema.org/medical/dicom/current/source/docbook/part03/part03.xml)
- [DICOM Part 6](https://dicom.nema.org/medical/dicom/current/source/docbook/part06/part06.xml)
- [DICOM Part 16](https://dicom.nema.org/medical/dicom/current/source/docbook/part16/part16.xml)

This notebook demonstrates how to parse out DICOM Part 3. 
1. Extract one IOD module table: A.36 Enhanced MR and its Reference tables
2. Extract all IOD module tables and their Reference tables


## 1.a Extract one IOD Module Table: A.36 Enhanced MR

In [2]:
import requests
from bs4 import BeautifulSoup

# Fetch XML content from the URL
url = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part03/part03.xml'
response = requests.get(url)
xml_content = response.content

# Parse the XML content
soup = BeautifulSoup(xml_content, 'xml')

/Users/wooyeonpark/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [16]:
import requests
import pandas as pd

# Find the table with label 'A.36-1'
table = soup.find('table', {'label': 'A.4-1'})

# Find the section containing the table and extract its title
section_title = soup.find('section', {'xml:id': table.parent['xml:id']}).find('title').text.strip()

# Extract the part of the title before "IOD"
iod_index = section_title.find(" IOD")
iod = section_title[:iod_index]

# Extract table headers
headers = [th.text.strip() for th in table.find_all('th')]

# Extract table rows
rows = []
current_ie = None
for tr in table.find_all('tr')[1:]:  # Skip the header row
    cells = tr.find_all(['td', 'th'])

    # Extract cell values
    cell_values = [cell.text.strip() for cell in cells]

    # Check if the cell values are complete (some cells may be missing due to rowspan)
    if len(cell_values) < len(headers):
        # Fill in the missing values based on the previous row
        for i in range(len(headers) - len(cell_values)):
            cell_values.insert(0, current_ie)

    # Store the IE value for the next iteration
    current_ie = cell_values[0]

    # Extract the reference value if available
    reference = None
    for cell in cells:
        xref = cell.find('xref', {'xrefstyle': 'select: labelnumber'})
        if xref:
            reference = xref['linkend']
            break

    # Insert the reference value into the correct position
    if reference:
        # Find the index of the cell containing the reference
        reference_index = next((idx for idx, val in enumerate(cell_values) if val == ''), None)
        if reference_index is not None:
            cell_values[reference_index] = reference

    # Insert the title of the table as the first element
    cell_values.insert(0, iod)

    rows.append(cell_values)

# Create a DataFrame
df_iod = pd.DataFrame(rows, columns=['IOD'] + headers)

In [17]:
df_iod

,IOD,IE,Module,Reference,Usage
0,MR Image,Patient,Patient,sect_C.7.1.1,M
1,MR Image,Patient,Clinical Trial Subject,sect_C.7.1.3,U
2,MR Image,Study,General Study,sect_C.7.2.1,M
3,MR Image,Study,Patient Study,sect_C.7.2.2,U
4,MR Image,Study,Clinical Trial Study,sect_C.7.2.3,U
5,MR Image,Series,General Series,sect_C.7.3.1,M
6,MR Image,Series,Clinical Trial Series,sect_C.7.3.2,U
7,MR Image,Frame of Reference,Frame of Reference,sect_C.7.4.1,M
8,MR Image,Equipment,General Equipment,sect_C.7.5.1,M
9,MR Image,Acquisition,General Acquisition,sect_C.7.10.1,M


## 1.b Extract Reference tables

In [7]:
# Find the mapping table between reference module from IOD table to reference table
rows = []

df_reference = df_iod[df_iod['Usage']=="M"]

for section_id in df_reference['Reference']:
    section_element = soup.find('section', {'xml:id': section_id})
    
    if section_element:
        tables = section_element.findChildren('table', recursive=False)
        # For each table, create a dictionary with section_id and table XML id and add it to the rows list
        for table in tables:
            if table.has_attr('xml:id'):
                row = {'section_id': section_id, 'table_xml_id': table['xml:id']}
                rows.append(row)
# Create a DataFrame from the rows list
table_df = pd.DataFrame(rows)
table_df

,section_id,table_xml_id
0,sect_C.7.1.1,table_C.7-1
1,sect_C.7.2.1,table_C.7-3
2,sect_C.7.3.1,table_C.7-5a
3,sect_C.7.4.1,table_C.7-6
4,sect_C.7.5.1,table_C.7-8
5,sect_C.7.10.1,table_C.7.10.1-1
6,sect_C.7.6.1,table_C.7-9
7,sect_C.7.6.2,table_C.7-10
8,sect_C.7.6.3,table_C.7-11a
9,sect_C.8.3.1,table_C.8-4


In [8]:
import pandas as pd
from bs4 import BeautifulSoup

def process_table(soup, table_xml_id):
    # Find the table using table_xml_id
    table = soup.find('table', {'xml:id': table_xml_id})
    table_xml_id = table.parent['xml:id']

    # Initialize lists to store extracted data
    attribute_names = []
    tags = []
    types = []
    attribute_descriptions = []
    cid = []

    # Extract data from the table rows
    for index, row in enumerate(table.find_all('tr')):
        columns = row.find_all('td')
        if len(columns) == 4:
            attribute_name = columns[0].text.strip()
            if attribute_name.startswith('>'):
                continue
            tag = columns[1].text.strip()
            type_ = columns[2].text.strip()
            description = columns[3].find_all('variablelist')

            defined_terms_dict = {}
            if description:
                for variablelist in description:
                    title = variablelist.title.text.strip().replace(':', '')
                    defined_terms = [term.text.strip() for term in variablelist.find_all('term')]
                    defined_terms_dict[title] = defined_terms
            
            attribute_names.append(attribute_name)
            tags.append(tag)
            types.append(type_)
            attribute_descriptions.append(defined_terms_dict)
            cid.append('')

        elif len(columns) == 2 and index != 0 and cid:
            first_col = columns[0].text.strip()
            if first_col.startswith('>>'):
                continue
            if first_col.startswith('>'):
                olink = columns[1].find('olink')
                if olink:
                    targetptr = olink.get('targetptr', '')
                    if 'CID_' in targetptr:
                        cid_value = targetptr.split('CID_')[-1]
                        cid[-1] += cid_value

    # Create a DataFrame from the extracted data
    data = {
        'section_id': table_xml_id,
        'Attribute Name': attribute_names,
        'Tag': tags,
        'Type': types,
        'Attribute Description': attribute_descriptions,
        'CID': cid
    }
    return pd.DataFrame(data)

In [9]:
reference_table_data = []
for table_xml_id in table_df['table_xml_id']:
    df = process_table(soup, table_xml_id)
    reference_table_data.append(df)

# combine all DataFrames into one
reference_tables = pd.concat(reference_table_data, ignore_index=True)

In [12]:
reference_tables

,section_id,Attribute Name,Tag,Type,Attribute Description,CID
0,sect_C.7.1.1,Patient's Name,"(0010,0010)",2,{},
1,sect_C.7.1.1,Patient ID,"(0010,0020)",2,{},
2,sect_C.7.1.1,Type of Patient ID,"(0010,0022)",3,"{'Defined Terms': ['TEXT', 'RFID', 'BARCODE']}",
3,sect_C.7.1.1,Patient's Birth Date,"(0010,0030)",2,{},
4,sect_C.7.1.1,Patient's Birth Date in Alternative Calendar,"(0010,0033)",3,{},
...,...,...,...,...,...,...
206,sect_C.12.1,Conversion Source Attributes Sequence,"(0020,9172)",1C,{},
207,sect_C.12.1,Content Qualification,"(0018,9004)",3,"{'Enumerated Values': ['PRODUCT', 'RESEARCH', ...",
208,sect_C.12.1,Private Data Element Characteristics Sequence,"(0008,0300)",3,{},
209,sect_C.12.1,Instance Origin Status,"(0400,0600)",3,"{'Enumerated Values': ['LOCAL', 'IMPORTED']}",


## 2.a Extract Module Tables for all IODs

In [25]:
from bs4 import BeautifulSoup

# Fetch XML content from the URL
url = 'https://dicom.nema.org/medical/dicom/current/source/docbook/part03/part03.xml'
response = requests.get(url)
xml_content = response.content

# Parse the XML content
soup = BeautifulSoup(xml_content, 'xml')

In [26]:
import re
from bs4 import BeautifulSoup
import pandas as pd

data = []

for section_element in soup.find_all('section', {'label': re.compile('A.')}):
    # Find all table elements within the section_element
    tables = section_element.find_all('table')
    for table in tables:
        # Find the caption element for the current table
        caption = table.find('caption')
        # Check if the caption exists and matches the regular expression
        if caption and re.search('IOD Modules', caption.text):
            # Extract the xml:id attribute from the table
            table_id = table.get('xml:id')  # Use .get() to avoid KeyError if the attribute is missing
            # Append the xml:id and caption text to the data list
            data.append((table_id, caption.text.strip()))

# Create a DataFrame from the collected data
df_iod_tables = pd.DataFrame(data, columns=['xml_id', 'iod'])
df_iod_tables = df_iod_tables.drop_duplicates().reset_index(drop=True)

In [27]:
df_iod_tables

,xml_id,iod
0,table_A.2-1,Computed Radiography Image IOD Modules
1,table_A.3-1,CT Image IOD Modules
2,table_A.4-1,MR Image IOD Modules
3,table_A.5-1,Nuclear Medicine Image IOD Modules
4,table_A.6-1,Ultrasound Image IOD Modules
...,...,...
166,table_A.87-1,Microscopy Bulk Simple Annotations IOD Modules
167,table_A.88.3-1,Inventory IOD Modules
168,table_A.89.3-1,Photoacoustic Image IOD Modules
169,table_A.90.1.3-1,Confocal Microscopy Image IOD Modules


In [31]:
import pandas as pd

first_table_headers = None
table_data = []  # A list to store the DataFrame for each table

for index, row in df_iod_tables.iterrows():
    xml_id = row['xml_id']
    caption = row['iod']

    # Find the table in 'soup' with the matching 'xml:id'
    table = soup.find('table', {'xml:id': xml_id})
    if table:
        # Extract table headers
        headers = [th.text.strip() for th in table.find_all('th')]
        # If it's the first table, store its headers
        if first_table_headers is None:
            first_table_headers = headers
        # For subsequent tables, check if the headers match the first table's headers
        elif headers != first_table_headers:
            # If headers don't match, skip this table
            continue

        # Reset the current_ie for each table to ensure data integrity
        current_ie = None  
        rows = []
        
        for tr in table.find_all('tr')[1:]:  # Skip the header row
            cells = tr.find_all(['td', 'th'])
            # Extract cell values
            cell_values = [cell.text.strip() for cell in cells]

            # Check if the cell values are complete (some cells may be missing due to rowspan)
            if len(cell_values) < len(headers):
                # Fill in the missing values based on the previous row
                for i in range(len(headers) - len(cell_values)):
                    cell_values.insert(0, current_ie)

            # Update current_ie based on the first cell value (the IE column)
            current_ie = cell_values[0]

            # Extract the reference value if available
            reference = None
            for cell in cells:
                xref = cell.find('xref', {'xrefstyle': 'select: labelnumber'})
                if xref:
                    reference = xref['linkend']
                    break

            # Insert the reference value into the correct position
            if reference:
                # Find the index of the cell containing the reference
                reference_index = next((idx for idx, val in enumerate(cell_values) if val == ''), None)
                if reference_index is not None:
                    cell_values[reference_index] = reference

            # Append the row data
            table_data.append([xml_id, caption] + cell_values)

# Create the final DataFrame
final_df = pd.DataFrame(table_data, columns=['xml_id', 'iod'] + first_table_headers)

In [35]:
final_df['Usage_code'] = final_df['Usage'].str[0]
final_df

,xml_id,iod,IE,Module,Reference,Usage,Usage_code
0,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M
1,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Clinical Trial Subject,sect_C.7.1.3,U,U
2,table_A.2-1,Computed Radiography Image IOD Modules,Study,General Study,sect_C.7.2.1,M,M
3,table_A.2-1,Computed Radiography Image IOD Modules,Study,Patient Study,sect_C.7.2.2,U,U
4,table_A.2-1,Computed Radiography Image IOD Modules,Study,Clinical Trial Study,sect_C.7.2.3,U,U
...,...,...,...,...,...,...,...
3264,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Cutaneous Confocal Microscopy Image Acquisitio...,sect_C.8.35.3,C - Required for cutaneous confocal microscopy,C
3265,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Optical Path,sect_C.8.12.5,M,M
3266,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,SOP Common,sect_C.12.1,M,M
3267,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Common Instance Reference,sect_C.12.2,U,U


In [32]:
final_df[final_df['xml_id']=="table_A.4-1"]

,xml_id,iod,IE,Module,Reference,Usage
47,table_A.4-1,MR Image IOD Modules,Patient,Patient,sect_C.7.1.1,M
48,table_A.4-1,MR Image IOD Modules,Patient,Clinical Trial Subject,sect_C.7.1.3,U
49,table_A.4-1,MR Image IOD Modules,Study,General Study,sect_C.7.2.1,M
50,table_A.4-1,MR Image IOD Modules,Study,Patient Study,sect_C.7.2.2,U
51,table_A.4-1,MR Image IOD Modules,Study,Clinical Trial Study,sect_C.7.2.3,U
52,table_A.4-1,MR Image IOD Modules,Series,General Series,sect_C.7.3.1,M
53,table_A.4-1,MR Image IOD Modules,Series,Clinical Trial Series,sect_C.7.3.2,U
54,table_A.4-1,MR Image IOD Modules,Frame of Reference,Frame of Reference,sect_C.7.4.1,M
55,table_A.4-1,MR Image IOD Modules,Equipment,General Equipment,sect_C.7.5.1,M
56,table_A.4-1,MR Image IOD Modules,Acquisition,General Acquisition,sect_C.7.10.1,M


In [92]:
final_df.groupby('Usage_code')['xml_id'].count()

Usage_code
C     314
M    1760
U    1179
Name: xml_id, dtype: int64

In [36]:
final_df.groupby('Usage_code')['xml_id'].count()

Usage_code
C     316
M    1768
U    1185
Name: xml_id, dtype: int64

In [37]:
final_df['xml_id'].nunique()

171

In [38]:
final_df['Reference'].nunique()

358

### Exception handling
It looks like we have a list of Reference tables that needed further adjustments. Upon investigation, I found that the standard for Reference tables above redirect to another table or tables. To handle this exception, I have manually collected the redirected References and created a dataframe as mapping_data.

In [39]:
#'sect_C.8.15.4': 'sect_C.8.2.2.1', 'sect_C.8.2.2.2', 'sect_C.8.2.2.3'
#'sect_C.11.11': 'sect_C.11.11.1'
#'sect_C.17.3': 'sect_C.17.3.3', 'sect_C.17.3.4'
#'sect_C.7.6.20': 'sect_C.10.12'
#'sect_C.28.1':'sect_C.10.9'
#'sect_C.29.3':'sect_C.29.3.1'
#'sect_C.36.18': 'sect_C.36.2.2.7', 'sect_C.36.2.2.8', 'sect_C.36.2.2.14'

mapping_data = {
    'Reference': ['sect_C.8.15.4', 'sect_C.8.15.4', 'sect_C.8.15.4', 'sect_C.11.11', 'sect_C.17.3', 'sect_C.17.3', 'sect_C.7.6.20', 'sect_C.28.1', 'sect_C.29.3', 'sect_C.36.18', 'sect_C.36.18', 'sect_C.36.18'],
    'Reference_adjusted': ['sect_C.8.2.2.1', 'sect_C.8.2.2.2', 'sect_C.8.2.2.3', 'sect_C.11.11.1', 'sect_C.17.3.3', 'sect_C.17.3.4', 'sect_C.10.12', 'sect_C.10.9', 'sect_C.29.3.1', 'sect_C.36.2.2.7', 'sect_C.36.2.2.8', 'sect_C.36.2.2.14']
}
mapping_data = pd.DataFrame(mapping_data)
mapping_data

,Reference,Reference_adjusted
0,sect_C.8.15.4,sect_C.8.2.2.1
1,sect_C.8.15.4,sect_C.8.2.2.2
2,sect_C.8.15.4,sect_C.8.2.2.3
3,sect_C.11.11,sect_C.11.11.1
4,sect_C.17.3,sect_C.17.3.3
5,sect_C.17.3,sect_C.17.3.4
6,sect_C.7.6.20,sect_C.10.12
7,sect_C.28.1,sect_C.10.9
8,sect_C.29.3,sect_C.29.3.1
9,sect_C.36.18,sect_C.36.2.2.7


In [40]:
final_df_expanded = pd.merge(final_df, mapping_data, on = 'Reference', how = 'left')
final_df_expanded['Reference_adjusted'] = final_df_expanded['Reference_adjusted'].fillna(final_df_expanded['Reference'])
final_df_expanded

,xml_id,iod,IE,Module,Reference,Usage,Usage_code,Reference_adjusted
0,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1
1,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Clinical Trial Subject,sect_C.7.1.3,U,U,sect_C.7.1.3
2,table_A.2-1,Computed Radiography Image IOD Modules,Study,General Study,sect_C.7.2.1,M,M,sect_C.7.2.1
3,table_A.2-1,Computed Radiography Image IOD Modules,Study,Patient Study,sect_C.7.2.2,U,U,sect_C.7.2.2
4,table_A.2-1,Computed Radiography Image IOD Modules,Study,Clinical Trial Study,sect_C.7.2.3,U,U,sect_C.7.2.3
...,...,...,...,...,...,...,...,...
3293,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Cutaneous Confocal Microscopy Image Acquisitio...,sect_C.8.35.3,C - Required for cutaneous confocal microscopy,C,sect_C.8.35.3
3294,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Optical Path,sect_C.8.12.5,M,M,sect_C.8.12.5
3295,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,SOP Common,sect_C.12.1,M,M,sect_C.12.1
3296,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Common Instance Reference,sect_C.12.2,U,U,sect_C.12.2


In [41]:
final_df_expanded[final_df_expanded['Reference']!= final_df_expanded['Reference_adjusted']]

,xml_id,iod,IE,Module,Reference,Usage,Usage_code,Reference_adjusted
889,table_A.33.1-1,Grayscale Softcopy Presentation State IOD Modules,Presentation State,Presentation State Relationship,sect_C.11.11,M,M,sect_C.11.11.1
917,table_A.33.2-1,Color Softcopy Presentation State IOD Modules,Presentation State,Presentation State Relationship,sect_C.11.11,M,M,sect_C.11.11.1
941,table_A.33.3-1,Pseudo-Color Softcopy Presentation State IOD M...,Presentation State,Presentation State Relationship,sect_C.11.11,M,M,sect_C.11.11.1
1007,table_A.33.6-1,XA/XRF Grayscale Softcopy Presentation State I...,Presentation State,Presentation State Relationship,sect_C.11.11,M,M,sect_C.11.11.1
1056,table_A.33.8-1,Variable Modality LUT Softcopy Presentation St...,Presentation State,Presentation State Relationship,sect_C.11.11,M,M,sect_C.11.11.1
...,...,...,...,...,...,...,...,...
2997,table_A.86.1.7-1,Robotic-Arm Radiation IOD Modules,RT Radiation,Robotic-Arm Delivery Device,sect_C.36.18,M,M,sect_C.36.2.2.8
2998,table_A.86.1.7-1,Robotic-Arm Radiation IOD Modules,RT Radiation,Robotic-Arm Delivery Device,sect_C.36.18,M,M,sect_C.36.2.2.14
3092,table_A.86.1.12-1,Robotic-Arm Radiation Record IOD Modules,RT Delivered Radiation,Robotic-Arm Delivery Device,sect_C.36.18,M,M,sect_C.36.2.2.7
3093,table_A.86.1.12-1,Robotic-Arm Radiation Record IOD Modules,RT Delivered Radiation,Robotic-Arm Delivery Device,sect_C.36.18,M,M,sect_C.36.2.2.8


In [43]:
final_df_expanded['Reference_adjusted'].nunique()

362

## 2.b Extract all Reference tables

In [42]:
# Find the mapping table between reference module from IOD table to reference table
# Find from the all IOD list for ALL usage
rows = []

df_reference = final_df_expanded['Reference_adjusted'].drop_duplicates()

for section_id in df_reference:
    section_element = soup.find('section', {'xml:id': section_id})
    
    if section_element:
        tables = section_element.findChildren('table', recursive=False)
        # For each table, create a dictionary with section_id and table XML id and add it to the rows list
        for table in tables:
            if table.has_attr('xml:id'):
                row = {'section_id': section_id, 'table_xml_id': table['xml:id']}
                rows.append(row)
# Create a DataFrame from the rows list
table_df = pd.DataFrame(rows)
table_df

,section_id,table_xml_id
0,sect_C.7.1.1,table_C.7-1
1,sect_C.7.1.3,table_C.7-2b
2,sect_C.7.2.1,table_C.7-3
3,sect_C.7.2.2,table_C.7-4a
4,sect_C.7.2.3,table_C.7-4b
...,...,...
371,sect_C.8.34.3,table_C.8.34.3-1
372,sect_C.8.34.4,table_C.8.34.4-1
373,sect_C.8.35.1,table_C.8.35.1-1
374,sect_C.8.35.3,table_C.8.35.3-1


In [52]:
table_df_agg = table_df.groupby('section_id')['table_xml_id'].agg(count = 'count').reset_index()
multiple_ref_tables = table_df_agg[table_df_agg['count']>1]['section_id']

In [61]:
dedup_table_xml = table_df[table_df['section_id'].isin(multiple_ref_tables)]
dedup_table_xml = dedup_table_xml.groupby('section_id').nth(1).reset_index(drop=True)
dedup_table_xml

,section_id,table_xml_id
0,sect_C.7.6.11,table_C.7-17a
1,sect_C.7.6.22,table_C.7.6.22-2
2,sect_C.11.1,table_C.11-1b
3,sect_C.11.2,table_C.11-2b
4,sect_C.7.9,table_C.7-22a
5,sect_C.8.7.3,table_C.8-28b
6,sect_C.8.11.4,table_C.8-71b
7,sect_C.8.7.10,table_C.8-35a
8,sect_C.8.7.11,table_C.8-36b
9,sect_C.23.2,table_C.23.2-2


In [62]:
table_df_new = table_df.merge(dedup_table_xml, on = 'section_id', how = 'left', suffixes=('', '_adj'))
table_df_new['table_xml_id_new'] = table_df_new['table_xml_id_adj'].combine_first(table_df_new['table_xml_id'])
table_df_new

,section_id,table_xml_id,table_xml_id_adj,table_xml_id_new
0,sect_C.7.1.1,table_C.7-1,NaN,table_C.7-1
1,sect_C.7.1.3,table_C.7-2b,NaN,table_C.7-2b
2,sect_C.7.2.1,table_C.7-3,NaN,table_C.7-3
3,sect_C.7.2.2,table_C.7-4a,NaN,table_C.7-4a
4,sect_C.7.2.3,table_C.7-4b,NaN,table_C.7-4b
...,...,...,...,...
371,sect_C.8.34.3,table_C.8.34.3-1,NaN,table_C.8.34.3-1
372,sect_C.8.34.4,table_C.8.34.4-1,NaN,table_C.8.34.4-1
373,sect_C.8.35.1,table_C.8.35.1-1,NaN,table_C.8.35.1-1
374,sect_C.8.35.3,table_C.8.35.3-1,NaN,table_C.8.35.3-1


In [68]:
table_df_new = table_df_new[['section_id', 'table_xml_id_new']].drop_duplicates().reset_index(drop=True)

In [69]:
# same function as above 
import pandas as pd
from bs4 import BeautifulSoup

def process_table(soup, table_xml_id):
    # Find the table using table_xml_id
    table = soup.find('table', {'xml:id': table_xml_id})
    table_xml_id = table.parent['xml:id']

    # Initialize lists to store extracted data
    attribute_names = []
    tags = []
    types = []
    attribute_descriptions = []
    cid = []

    # Extract data from the table rows
    for index, row in enumerate(table.find_all('tr')):
        columns = row.find_all('td')
        if len(columns) == 4:
            attribute_name = columns[0].text.strip()
            if attribute_name.startswith('>'):
                continue
            tag = columns[1].text.strip()
            type_ = columns[2].text.strip()
            description = columns[3].find_all('variablelist')

            defined_terms_dict = {}
            if description:
                for variablelist in description:
                    title = variablelist.title.text.strip().replace(':', '')
                    defined_terms = [term.text.strip() for term in variablelist.find_all('term')]
                    defined_terms_dict[title] = defined_terms
            
            attribute_names.append(attribute_name)
            tags.append(tag)
            types.append(type_)
            attribute_descriptions.append(defined_terms_dict)
            cid.append('')

        elif len(columns) == 2 and index != 0 and cid:
            first_col = columns[0].text.strip()
            if first_col.startswith('>>'):
                continue
            if first_col.startswith('>'):
                olink = columns[1].find('olink')
                if olink:
                    targetptr = olink.get('targetptr', '')
                    if 'CID_' in targetptr:
                        cid_value = targetptr.split('CID_')[-1]
                        cid[-1] += cid_value

    # Create a DataFrame from the extracted data
    data = {
        'section_id': table_xml_id,
        'Attribute Name': attribute_names,
        'Tag': tags,
        'Type': types,
        'Attribute Description': attribute_descriptions,
        'CID': cid
    }
    return pd.DataFrame(data)

In [71]:
reference_table_data = []
for table_xml_id in table_df_new['table_xml_id_new']:
    df = process_table(soup, table_xml_id)
    reference_table_data.append(df)

# combine all DataFrames into one
reference_tables = pd.concat(reference_table_data, ignore_index=True)

In [74]:
# subset of tags based on type = required
reference_tables[reference_tables['Type']=='1']

,section_id,Attribute Name,Tag,Type,Attribute Description,CID
34,sect_C.7.1.3,Clinical Trial Sponsor Name,"(0012,0010)",1,{},
35,sect_C.7.1.3,Clinical Trial Protocol ID,"(0012,0020)",1,{},
48,sect_C.7.2.1,Study Instance UID,"(0020,000D)",1,{},
100,sect_C.7.3.1,Modality,"(0008,0060)",1,{},
101,sect_C.7.3.1,Series Instance UID,"(0020,000E)",1,{},
...,...,...,...,...,...,...
2653,sect_C.8.35.1,Tissue Location,"(0048,0115)",1,"{'Enumerated Values': ['INVIVO', 'EXVIVO']}",
2660,sect_C.8.35.2,Imaged Volume Width,"(0048,0001)",1,{},
2661,sect_C.8.35.2,Imaged Volume Height,"(0048,0002)",1,{},
2662,sect_C.8.35.2,Imaged Volume Depth,"(0048,0003)",1,{},


In [75]:
# subset of tags with attribute description
reference_tables[reference_tables['Attribute Description']!= {}]

,section_id,Attribute Name,Tag,Type,Attribute Description,CID
2,sect_C.7.1.1,Type of Patient ID,"(0010,0022)",3,"{'Defined Terms': ['TEXT', 'RFID', 'BARCODE']}",
7,sect_C.7.1.1,Patient's Sex,"(0010,0040)",2,"{'Enumerated Values': ['M', 'F', 'O']}",
9,sect_C.7.1.1,Quality Control Subject,"(0010,0200)",3,"{'Enumerated Values': ['YES', 'NO']}",
31,sect_C.7.1.1,Patient Identity Removed,"(0012,0062)",3,"{'Enumerated Values': ['YES', 'NO']}",
79,sect_C.7.2.2,Smoking Status,"(0010,21A0)",3,"{'Enumerated Values': ['YES', 'NO', 'UNKNOWN']}",
...,...,...,...,...,...,...
2651,sect_C.8.35.1,Lossy Image Compression,"(0028,2110)",1,"{'Enumerated Values': ['00', '01']}",
2652,sect_C.8.35.1,Confocal Mode,"(0048,0114)",1,"{'Enumerated Values': ['REFLECTANCE', 'FLUORES...",
2653,sect_C.8.35.1,Tissue Location,"(0048,0115)",1,"{'Enumerated Values': ['INVIVO', 'EXVIVO']}",
2656,sect_C.8.35.3,Field of View Shape,"(0018,1147)",2,{'Defined Terms': ['RECTANGLE']},


In [76]:
# subset of tags with CIDs
reference_tables[reference_tables['CID']!='']

,section_id,Attribute Name,Tag,Type,Attribute Description,CID
15,sect_C.7.1.1,Ethnic Group Code Sequence,"(0010,2161)",3,{},6099
18,sect_C.7.1.1,Patient Species Code Sequence,"(0010,2202)",1C,{},7454
20,sect_C.7.1.1,Patient Breed Code Sequence,"(0010,2293)",2C,{},7480
33,sect_C.7.1.1,De-identification Method Code Sequence,"(0012,0064)",1C,{},7050
64,sect_C.7.2.1,Requesting Service Code Sequence,"(0032,1034)",3,{},7030
...,...,...,...,...,...,...
2635,sect_C.8.34.2,Illumination Type Code Sequence,"(0022,0016)",3,{},11001
2637,sect_C.8.34.2,Acoustic Coupling Medium Code Sequence,"(0018,982A)",2C,{},11002
2639,sect_C.8.34.3,Transducer Geometry Code Sequence,"(0018,980D)",1,{},12033
2641,sect_C.8.34.3,Transducer Technology Sequence,"(0018,9831)",3,{},11003


In [77]:
# create one dataframe and export the file
mapping_table = pd.merge(final_df_expanded, reference_tables, left_on = 'Reference_adjusted', right_on = 'section_id', how = 'left')
mapping_table

,xml_id,iod,IE,Module,Reference,Usage,Usage_code,Reference_adjusted,section_id,Attribute Name,Tag,Type,Attribute Description,CID
0,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Patient's Name,"(0010,0010)",2,{},
1,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Patient ID,"(0010,0020)",2,{},
2,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Type of Patient ID,"(0010,0022)",3,"{'Defined Terms': ['TEXT', 'RFID', 'BARCODE']}",
3,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Patient's Birth Date,"(0010,0030)",2,{},
4,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,sect_C.7.1.1,Patient's Birth Date in Alternative Calendar,"(0010,0033)",3,{},
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38766,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,SOP Common,sect_C.12.1,M,M,sect_C.12.1,sect_C.12.1,Instance Origin Status,"(0400,0600)",3,"{'Enumerated Values': ['LOCAL', 'IMPORTED']}",
38767,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,SOP Common,sect_C.12.1,M,M,sect_C.12.1,sect_C.12.1,Barcode Value,"(2200,0005)",3,{},
38768,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Common Instance Reference,sect_C.12.2,U,U,sect_C.12.2,sect_C.12.2,Referenced Series Sequence,"(0008,1115)",1C,{},
38769,table_A.90.2.3-1,Confocal Microscopy Tiled Pyramidal Image IOD ...,Image,Common Instance Reference,sect_C.12.2,U,U,sect_C.12.2,sect_C.12.2,Studies Containing Other Referenced Instances ...,"(0008,1200)",1C,{},


In [86]:
mapping_table['Tag'] = mapping_table['Tag'].str.replace(r'[(),]', '', regex = True)
mapping_table = mapping_table.drop(columns=['section_id'])
mapping_table.head()

,xml_id,iod,IE,Module,Reference,Usage,Usage_code,Reference_adjusted,Attribute Name,Tag,Type,Attribute Description,CID
0,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient's Name,00100010,2,{},
1,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient ID,00100020,2,{},
2,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Type of Patient ID,00100022,3,"{'Defined Terms': ['TEXT', 'RFID', 'BARCODE']}",
3,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient's Birth Date,00100030,2,{},
4,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient's Birth Date in Alternative Calendar,00100033,3,{},


## Add SOP Class UID from Part 4
*Extract Part 4 ("DICOM_P4_harvest_SOP) before running*

In [88]:
mapping_table['short_iod'] = mapping_table['iod'].str.replace(' IOD Modules', '')

In [89]:
sop = pd.read_csv("./files/DICOM Standard/part4_sop_class.csv")
sop['Truncated SOP Class Name'] = sop['Truncated SOP Class Name'].str.replace(' Storage', '')

In [90]:
update_dict = {"Cardiac Electrophysiology Waveform Storage": "Basic Cardiac EP",
 "Ambulatory ECG Waveform Storage": "Ambulatory ECG",
 "General ECG Waveform Storage": "General ECG",
 "12-lead ECG Waveform Storage": "12-Lead ECG"
}

def update_truncated_sop_class_name(row):
    if row['SOP Class Name'] in update_dict:
        return update_dict[row['SOP Class Name']]
    return row['Truncated SOP Class Name']

# Apply the function to update the column
sop['Truncated SOP Class Name'] = sop.apply(update_truncated_sop_class_name, axis=1)

In [91]:
sop[sop['SOP Class Name'].isin(update_dict.keys())]

,SOP Class Name,SOP Class UID,Truncated SOP Class Name
24,12-lead ECG Waveform Storage,1.2.840.10008.5.1.4.1.1.9.1.1,12-Lead ECG
25,General ECG Waveform Storage,1.2.840.10008.5.1.4.1.1.9.1.2,General ECG
26,Ambulatory ECG Waveform Storage,1.2.840.10008.5.1.4.1.1.9.1.3,Ambulatory ECG
29,Cardiac Electrophysiology Waveform Storage,1.2.840.10008.5.1.4.1.1.9.3.1,Basic Cardiac EP


In [92]:
# Truncated SOP Class Name, SOP Class UID
data = {
    "SOP Class Name": [
        "Implantation Plan SR Document Storage",
        "RT Treatment Summary Record Storage",
        "RT Ion Plan Storage",
        "RT Ion Beams Treatment Record Storage",
        "RT Beams Delivery Instruction Storage",
        "Generic Implant Template Storage",
        "Implant Assembly Template Storage",
        "Implant Template Group Storage"
        ],
    "SOP Class UID": [
        "1.2.840.10008.5.1.4.1.1.88.70",
        "1.2.840.10008.5.1.4.1.1.481.7",
        "1.2.840.10008.5.1.4.1.1.481.8",
        "1.2.840.10008.5.1.4.1.1.481.9",
        "1.2.840.10008.5.1.4.34.7",
        "1.2.840.10008.5.1.4.43.1",
        "1.2.840.10008.5.1.4.44.1",
        "1.2.840.10008.5.1.4.45.1"
        ],
    "Truncated SOP Class Name": [
        "Implantation Plan SR Document",
        "RT Treatment Summary Record",
        "RT Ion Plan",
        "RT Ion Beams Treatment Record",
        "RT Beams Delivery Instruction",
        "Generic Implant Template",
        "Implant Assembly Template",
        "Implant Template Group"
        ]
}

data_input = pd.DataFrame(data)

# Concatenate the original DataFrame with the new rows DataFrame
sop = pd.concat([sop, data_input], ignore_index=True)

In [93]:
mapping_table = mapping_table.merge(sop, left_on = 'short_iod', right_on = 'Truncated SOP Class Name', how = 'left')
mapping_table.head()

,xml_id,iod,IE,Module,Reference,Usage,Usage_code,Reference_adjusted,Attribute Name,Tag,Type,Attribute Description,CID,short_iod,SOP Class Name,SOP Class UID,Truncated SOP Class Name
0,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient's Name,00100010,2,{},,Computed Radiography Image,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,Computed Radiography Image
1,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient ID,00100020,2,{},,Computed Radiography Image,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,Computed Radiography Image
2,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Type of Patient ID,00100022,3,"{'Defined Terms': ['TEXT', 'RFID', 'BARCODE']}",,Computed Radiography Image,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,Computed Radiography Image
3,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient's Birth Date,00100030,2,{},,Computed Radiography Image,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,Computed Radiography Image
4,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient's Birth Date in Alternative Calendar,00100033,3,{},,Computed Radiography Image,Computed Radiography Image Storage,1.2.840.10008.5.1.4.1.1.1,Computed Radiography Image


In [95]:
mapping_table = mapping_table.drop(columns = ['short_iod', 'Truncated SOP Class Name', 'SOP Class Name'])
mapping_table.head()

,xml_id,iod,IE,Module,Reference,Usage,Usage_code,Reference_adjusted,Attribute Name,Tag,Type,Attribute Description,CID,SOP Class UID
0,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient's Name,00100010,2,{},,1.2.840.10008.5.1.4.1.1.1
1,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient ID,00100020,2,{},,1.2.840.10008.5.1.4.1.1.1
2,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Type of Patient ID,00100022,3,"{'Defined Terms': ['TEXT', 'RFID', 'BARCODE']}",,1.2.840.10008.5.1.4.1.1.1
3,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient's Birth Date,00100030,2,{},,1.2.840.10008.5.1.4.1.1.1
4,table_A.2-1,Computed Radiography Image IOD Modules,Patient,Patient,sect_C.7.1.1,M,M,sect_C.7.1.1,Patient's Birth Date in Alternative Calendar,00100033,3,{},,1.2.840.10008.5.1.4.1.1.1


In [96]:
mapping_table.to_pickle('./files/DICOM Standard/part3_mapping.pkl')